In [30]:
import pandas as pd
import sqlite3

In [32]:
data=pd.read_html('https://www.moneycontrol.com/markets/seasonality-analysis/scan-stocks?id=7&type=indices&ex=N')
data[0]

,Symbol,Total Year,Total Positive Year,%Positive,Max Positive Change(%),Min Positive Change(%),Average Positive Change(%)
0,Piramal Pharma,3.0,3.0,100.00,12.93,2.21,9.03
1,Latent View Analytics,4.0,4.0,100.00,11.39,2.59,6.48
2,Adani Wilmar,4.0,4.0,100.00,14.74,0.48,5.79
3,Devyani International,4.0,4.0,100.00,15.19,1.01,6.89
4,Krishna Institute of Medical Sciences,5.0,5.0,100.00,27.28,3.00,9.53
...,...,...,...,...,...,...,...
453,Fertilisers and Chemicals Travancore,17.0,5.0,29.41,17.79,2.17,7.54
454,Alok Industries,17.0,5.0,29.41,32.73,0.78,10.06
455,ABB India,17.0,5.0,29.41,18.92,2.93,6.65
456,Aditya Birla Capital,8.0,2.0,25.00,18.96,9.98,14.47


In [34]:
# Display number of tables
print(len(data))
df = data[0]
print(df.head())
print(df.columns)

1
                                  Symbol  Total Year  Total Positive Year  \
0                         Piramal Pharma         3.0                  3.0   
1                  Latent View Analytics         4.0                  4.0   
2                           Adani Wilmar         4.0                  4.0   
3                  Devyani International         4.0                  4.0   
4  Krishna Institute of Medical Sciences         5.0                  5.0   

   %Positive  Max Positive Change(%)  Min Positive Change(%)  \
0      100.0                   12.93                    2.21   
1      100.0                   11.39                    2.59   
2      100.0                   14.74                    0.48   
3      100.0                   15.19                    1.01   
4      100.0                   27.28                    3.00   

   Average Positive Change(%)  
0                        9.03  
1                        6.48  
2                        5.79  
3                     

In [36]:
# Connect to SQLite database
conn = sqlite3.connect('financial_data.db')
c=conn.cursor()

In [38]:
#create a wrapper function (renaming it)
def rsql(query):
    return pd.read_sql_query(query,conn)

In [40]:
c.execute("""
CREATE TABLE IF NOT EXISTS performance_table (
    Symbol TEXT,
    Company TEXT,
    Industry TEXT,
    Sector TEXT,
    Max_Positive_Change REAL,
    Average_Positive_Change REAL,
    Min_Positive_Change REAL,
    Percent_Positive REAL,
    Total_Positive_Year INT
)
""")

In [42]:
# Insert scraped data into SQL table
df.to_sql('performance_table', conn, if_exists='replace', index=False)
print(df.columns)

Index(['Symbol', 'Total Year', 'Total Positive Year', '%Positive',
       'Max Positive Change(%)', 'Min Positive Change(%)',
       'Average Positive Change(%)'],
      dtype='object')


In [44]:
rsql('SELECT * FROM performance_table LIMIT 7')

,Symbol,Total Year,Total Positive Year,%Positive,Max Positive Change(%),Min Positive Change(%),Average Positive Change(%)
0,Piramal Pharma,3.0,3.0,100.0,12.93,2.21,9.03
1,Latent View Analytics,4.0,4.0,100.0,11.39,2.59,6.48
2,Adani Wilmar,4.0,4.0,100.0,14.74,0.48,5.79
3,Devyani International,4.0,4.0,100.0,15.19,1.01,6.89
4,Krishna Institute of Medical Sciences,5.0,5.0,100.0,27.28,3.00,9.53
5,UTI Asset Management Company,5.0,5.0,100.0,22.03,2.17,8.88
6,Home First Finance Company India,5.0,5.0,100.0,11.29,0.40,6.80


In [46]:
def run_query(query):
    return pd.read_sql_query(query, conn)

In [48]:
# 1. Stocks with Max Positive Change > 10%
run_query("""
SELECT Symbol, `Max Positive Change(%)`
FROM performance_table
WHERE `Max Positive Change(%)` > 10
""")

,Symbol,Max Positive Change(%)
0,Piramal Pharma,12.93
1,Latent View Analytics,11.39
2,Adani Wilmar,14.74
3,Devyani International,15.19
4,Krishna Institute of Medical Sciences,27.28
...,...,...
441,Fertilisers and Chemicals Travancore,17.79
442,Alok Industries,32.73
443,ABB India,18.92
444,Aditya Birla Capital,18.96


In [50]:
#2. Companies with average positive change > 8%
run_query("""
SELECT Symbol, AVG(`Average Positive Change(%)`) AS AvgChange
FROM performance_table
GROUP BY Symbol
HAVING AvgChange > 8
""")

,Symbol,AvgChange
0,360 ONE WAM,13.80
1,3M India,9.22
2,ACC,9.12
3,APL Apollo Tubes,11.02
4,AU Small Finance Bank,18.02
...,...,...
310,Zen Technologies,22.13
311,Zensar Technologies,22.55
312,Zomato,9.63
313,Zydus Lifesciences,9.76


In [52]:
#Q3. Number of companies 

run_query("""
SELECT COUNT(*) AS CompanyCount
FROM performance_table
""")

,CompanyCount
0,458


In [54]:
#4 Total number of positive years across all companies
run_query("""
SELECT SUM(`Total Positive Year`) AS TotalPositiveYears
FROM performance_table
""")

,TotalPositiveYears
0,3869.0


In [56]:
# Q5. Average %Positive for all companies
run_query("""
SELECT AVG(`%Positive`) AS AvgPositive
FROM performance_table
""")

,AvgPositive
0,61.731943


In [58]:
#Q6. Top 5 companies with highest Average Positive Change
run_query("""
SELECT Symbol, `Average Positive Change(%)`
FROM performance_table
ORDER BY `Average Positive Change(%)` DESC
LIMIT 5
""")

,Symbol,Average Positive Change(%)
0,Lloyds Metals and Energy,13284.04
1,Laurus Labs,28.77
2,Rattanindia Enterprises,28.26
3,Adani Green Energy Limited,27.06
4,Inox Wind,26.84


In [60]:
#Q7. Bottom 10 performers by Min Positive Change
run_query("""
SELECT Symbol, `Min Positive Change(%)`
FROM performance_table
ORDER BY `Min Positive Change(%)` ASC
LIMIT 10
""")

,Symbol,Min Positive Change(%)
0,Oil and Natural Gas Corporation,0.00
1,Eris Lifesciences,0.01
2,Capri Global Capital,0.01
3,Yes Bank,0.01
4,Kotak Mahindra Bank,0.02
5,UPL,0.02
6,AIA Engineering,0.02
7,Neuland Laboratories,0.03
8,Adani Green Energy Limited,0.03
9,Prestige Estates Projects,0.03


In [62]:
#Q8. Show Symbol and Average Positive Change, label as 'AvgGain'
run_query("""
SELECT Symbol, `Average Positive Change(%)` AS AvgGain
FROM performance_table
""")


,Symbol,AvgGain
0,Piramal Pharma,9.03
1,Latent View Analytics,6.48
2,Adani Wilmar,5.79
3,Devyani International,6.89
4,Krishna Institute of Medical Sciences,9.53
...,...,...
453,Fertilisers and Chemicals Travancore,7.54
454,Alok Industries,10.06
455,ABB India,6.65
456,Aditya Birla Capital,14.47


In [64]:
# Q9. Companies with Average Positive Change between 5% and 10%
run_query("""
SELECT Symbol, `Average Positive Change(%)`
FROM performance_table
WHERE `Average Positive Change(%)` BETWEEN 5 AND 10
""")

,Symbol,Average Positive Change(%)
0,Piramal Pharma,9.03
1,Latent View Analytics,6.48
2,Adani Wilmar,5.79
3,Devyani International,6.89
4,Krishna Institute of Medical Sciences,9.53
...,...,...
227,Praj Industries,8.15
228,Jammu and Kashmir Bank,8.26
229,Tata Investment Corporation,7.84
230,Fertilisers and Chemicals Travancore,7.54


In [66]:
#10. List all distinct Symbols (since no Industry column)
run_query("""
SELECT DISTINCT Symbol
FROM performance_table
""")

,Symbol
0,Piramal Pharma
1,Latent View Analytics
2,Adani Wilmar
3,Devyani International
4,Krishna Institute of Medical Sciences
...,...
453,Fertilisers and Chemicals Travancore
454,Alok Industries
455,ABB India
456,Aditya Birla Capital


In [68]:
#Q11. Insert a new company’s data (mock entry)
# Use cursor.execute for INSERT
cur = conn.cursor()

cur.execute("""
INSERT INTO performance_table (Symbol, `Total Year`, `Total Positive Year`, `%Positive`, `Max Positive Change(%)`, `Min Positive Change(%)`, `Average Positive Change(%)`)
VALUES ('Test Pharma', 2, 2, 100.0, 11.5, 5.2, 8.3)
""")

conn.commit()

# Check inserted
run_query("""
SELECT *
FROM performance_table
WHERE Symbol = 'Test Pharma'
""")

,Symbol,Total Year,Total Positive Year,%Positive,Max Positive Change(%),Min Positive Change(%),Average Positive Change(%)
0,Test Pharma,2.0,2.0,100.0,11.5,5.2,8.3


In [70]:
# Q12. Update Max Positive Change for Piramal Pharma

cur.execute("""
UPDATE performance_table
SET `Max Positive Change(%)` = 15.2
WHERE Symbol = 'Piramal Pharma'
""")

conn.commit()

# Check updated row
run_query("""
SELECT Symbol, `Max Positive Change(%)`
FROM performance_table
WHERE Symbol = 'Piramal Pharma'
""")


,Symbol,Max Positive Change(%)
0,Piramal Pharma,15.2
